In [1]:
from generate import *
from encode import *
from DataReader import *
from RNN import *

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
gen_data_and_vocab()

Processing: 0-alt.atheism
Processing: 1-comp.graphics
Processing: 2-comp.os.ms-windows.misc
Processing: 3-comp.sys.ibm.pc.hardware
Processing: 4-comp.sys.mac.hardware
Processing: 5-comp.windows.x
Processing: 6-misc.forsale
Processing: 7-rec.autos
Processing: 8-rec.motorcycles
Processing: 9-rec.sport.baseball
Processing: 10-rec.sport.hockey
Processing: 11-sci.crypt
Processing: 12-sci.electronics
Processing: 13-sci.med
Processing: 14-sci.space
Processing: 15-soc.religion.christian
Processing: 16-talk.politics.guns
Processing: 17-talk.politics.mideast
Processing: 18-talk.politics.misc
Processing: 19-talk.religion.misc
Processing: 0-alt.atheism
Processing: 1-comp.graphics
Processing: 2-comp.os.ms-windows.misc
Processing: 3-comp.sys.ibm.pc.hardware
Processing: 4-comp.sys.mac.hardware
Processing: 5-comp.windows.x
Processing: 6-misc.forsale
Processing: 7-rec.autos
Processing: 8-rec.motorcycles
Processing: 9-rec.sport.baseball
Processing: 10-rec.sport.hockey
Processing: 11-sci.crypt
Processing

In [3]:
encode_data('w2v/20news-train-raw.txt', 'w2v/vocab-raw.txt')
encode_data('w2v/20news-test-raw.txt', 'w2v/vocab-raw.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [4]:
def train_and_evaluate_RNN():
    with open('w2v/vocab-raw.txt') as f:
        vocab_size = len(f.read().splitlines())
    tf.set_random_seed(2024)

    rnn = RNN(
        vocab_size=vocab_size,
        embedding_size=300,
        lstm_size=50,
        batch_size=50
    )
    predicted_labels, loss = rnn.build_graph()
    train_op = rnn.trainer(loss=loss, learning_rate=0.01)
    
    with tf.Session() as sess:
        train_data_reader = DataReader(
            data_path='w2v/20news-train-encoded.txt',
            batch_size=50
        )
        test_data_reader = DataReader(
            data_path='w2v/20news-test-encoded.txt',
            batch_size=50
        )
        step = 0
        MAX_STEP = 1000
        sess.run(tf.global_variables_initializer())
        
        while step < MAX_STEP:
            next_train_batch = train_data_reader.next_batch()
            train_data, train_labels, train_sentence_lengths, train_final_tokens = next_train_batch
            plabels_eval, loss_eval, _ = sess.run(
                [predicted_labels, loss, train_op],
                feed_dict={
                    rnn._data: train_data,
                    rnn._labels: train_labels,
                    rnn._sentence_lengths: train_sentence_lengths,
                    rnn._final_tokens: train_final_tokens
                }
            )
            step += 1
            if step % 20 == 0:
                print(f'loss : {loss_eval}')

            if train_data_reader._batch_id == 0:
                num_true_preds = 0
                while True:
                    next_test_batch = test_data_reader.next_batch()
                    test_data, test_labels, test_sentence_lengths, test_final_tokens = next_test_batch
                    test_plabels_eval = sess.run(
                        predicted_labels,
                        feed_dict={
                            rnn._data: test_data,
                            rnn._labels: test_labels,
                            rnn._sentence_lengths: test_sentence_lengths,
                            rnn._final_tokens: test_final_tokens
                        }
                    )
                    
                    matches = np.equal(test_plabels_eval, test_labels)
                    num_true_preds += np.sum(matches.astype(float))
                    if test_data_reader._batch_id == 0:
                        break
                    
                print(f'Epoch: {train_data_reader._num_epoch}')
                print(f'Accuracy on test data: {num_true_preds * 100. / len(test_data_reader._data)}')
                

In [5]:
train_and_evaluate_RNN()

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/Users/dayo/Documents/ML-Lab-Training/session 4/RNN.py:33: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self._lstm_size)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



2024-11-16 17:15:56.662821: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2024-11-16 17:15:57.312619: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


loss : 7.144278060877696e-05
loss : 0.06548171490430832
loss : 8.959388732910156
loss : 0.01862339675426483
loss : 0.38219165802001953
loss : 5.000552177429199
loss : 2.5724804401397705
loss : 0.7893913984298706
loss : 4.1125712394714355
loss : 4.122959136962891
loss : 6.1617913246154785
Epoch: 1
Accuracy on test data: 8.47052575677111
loss : 2.945035696029663
loss : 2.304654359817505
loss : 2.212629795074463
loss : 1.8416088819503784
loss : 1.8866755962371826
loss : 1.7693623304367065
loss : 1.6243418455123901
loss : 1.565423846244812
loss : 1.4383996725082397
loss : 1.4029066562652588
loss : 1.3367342948913574
Epoch: 2
Accuracy on test data: 71.0833775889538
loss : 1.0192599296569824
loss : 0.926663339138031
loss : 0.9314233660697937
loss : 0.8637201189994812
loss : 0.8783875107765198
loss : 0.635991096496582
loss : 0.6612381935119629
loss : 0.606234610080719
loss : 0.5575686693191528
loss : 0.4075246751308441
loss : 0.5308899879455566
Epoch: 3
Accuracy on test data: 74.4689325544344